# Setup kaggle environment

In [1]:
import shutil
import sys
import os

( os.remove('/kaggle/bin'), shutil.rmtree('/kaggle/dlml') ) if os.path.exists('/kaggle/bin') else None
if not os.path.exists('/kaggle/bin'):
    os.environ['PATH'] += ':/kaggle/bin'
    sys.path.append('/kaggle/dlml/lib')
    
    if not os.path.exists('/root/.ssh'):
        os.mkdir('/root/.ssh')
        shutil.copyfile('/kaggle/input/kaggle-init/config','/root/.ssh/config')

    !sh /kaggle/input/kaggle-init/init   # git clone dlml

%run -i /kaggle/dlml/init/kaggle_init    # %run 可以不加 .py 後綴； Pure python code
!source /kaggle/dlml/init/kaggle_sh_init
%rehashx

Agent pid 68
Identity added: /root/.ssh/id_rsa (loaxsc)
Cloning into 'dlml'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 136 (delta 45), reused 98 (delta 21), pack-reused 0
Receiving objects: 100% (136/136), 19.54 KiB | 6.51 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [ ]:
exec_bash("""
mkdir -p /kaggle/dlml/lib
mv /kaggle/dlml/ipython_exit.py /kaggle/dlml/lib
ls /kaggle/dlml/lib
""")

### kaggle-init/init (download dlml)

### edit dlml/init/kaggle_sh_init

In [ ]:
%%writefile /kaggle/dlml/init/kaggle_sh_init

cp /kaggle/input/kaggle-init/client_secrets.json /bin
ln /bin/gd-upload-file /kaggle/dlml/bin/gd-upload-file

### edit dlml/init/kaggle_init

In [ ]:
%%writefile /kaggle/dlml/init/kaggle_init
#!/bin/python

import shutil
import os

from ipython_exit import exit

# get_ipython().run_line_magic('rehashx', '')

get_ipython().run_line_magic('alias', 'la ls -AF --color')

with open('/kaggle/dlml/init/.shinit','r') as f:
    sh_rc = f.read()

def exec_bash(code):         # %%bash 的輸出沒有顏色
    with open('/dev/shm/xxx___', 'w') as f:
        f.write('shopt -s expand_aliases\n%s' % sh_rc + code)
    get_ipython().system('source /dev/shm/xxx___')

def exec_sh(code):         # %%bash 的輸出沒有顏色
    with open('/dev/shm/xxx___', 'w') as f:
        f.write(sh_rc + code)
    get_ipython().system('sh /dev/shm/xxx___')
    
from IPython.display import FileLink
def get_file_link(path):
    filename = path.split('/')[-1]
    abspath = os.path.abspath(path)
    if 'working' not in os.path.abspath(abspath):
        print('copy to /kaggle/working/%s' % filename)
        shutil.copy(abspath, '.') # 無法用 symlink，已測試過
        return FileLink(filename)
    else:
        return FileLink(path)

### edit dlml/init/.shinit

In [ ]:
%%writefile /kaggle/dlml/init/.shinit

alias ls='ls --color=auto'
alias grep='grep --color=auto'
alias fgrep='fgrep --color=auto'
alias egrep='egrep --color=auto'

alias ll='ls -alF'
alias la='ls -A'
alias l='ls -CF'

In [ ]:
%run -i /kaggle/dlml/init/kaggle_init
exec_bash("""
la /kaggle
""")

In [ ]:
%github-push-kaggle-config-files 'fix kaggle_sh_init error'

# Install packages

In [ ]:
# gdown --id 1amMgpOEIMz_T1D-dPnXgucjhRg8xrEOc
# unzip -qu faster-rcnn.pytorch.zip

exec_bash("""
sh /kaggle/input/voc2007/init

cd /kaggle
echo 'pip install ...'
pip install gdown easydict 'git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI'
gdown --id 1eSmht15mrqbhPMjLTlxwcSEew2wotpZ3
tar -jxf faster-rcnn.pytorch.tar.bz2
rm faster-rcnn.pytorch.tar.bz2

cd /kaggle/faster-rcnn.pytorch/
#pip install -r requirements.txt # install easydict
if [ "$(which nvidia-smi)" ]; then
    cd /kaggle/faster-rcnn.pytorch/lib
    python setup.py develop
else
    echo '\nNot in GPU mode, exit !'
    exit
fi

cd /kaggle/faster-rcnn.pytorch/
mkdir -p data data/VOCdevkit2007 data/pretrained_model
ln -snf /kaggle/VOC2007 data/VOCdevkit2007/VOC2007
ln -snf /kaggle/working models

cd data/pretrained_model/
ln -snf /kaggle/input/od-models/resnet101_caffe.pth resnet101_caffe.pth
ln -snf /kaggle/input/vgg16-caffe/vgg16_caffe.pth vgg16_caffe.pth
""")

# copying build/lib.linux-x86_64-3.7/model/_C.cpython-37m-x86_64-linux-gnu.so -> model
# Creating /opt/conda/lib/python3.7/site-packages/faster-rcnn.egg-link (link to .)
# Adding faster-rcnn 0.1 to easy-install.pth file

# Installed /kaggle/faster-rcnn.pytorch/lib
# Processing dependencies for faster-rcnn==0.1
# Finished processing dependencies for faster-rcnn==0.1

In [ ]:
# error in _mask
# data/cache/*.pkl
exec_bash("""
cd /kaggle/faster-rcnn.pytorch
python trainval_net.py \
            --dataset pascal_voc --net res101  \
            --bs 4 \
            --lr "4e-3" --lr_decay_step 8 \
            --cuda --epochs 1
""")

In [ ]:
exec_bash("""
cd /kaggle
tar -jcf faster-rcnn.pytorch.tar.bz2 faster-rcnn.pytorch
""")
get_file_link('/kaggle/faster-rcnn.pytorch.tar.bz2')